In [1]:
#!g1.1
!git clone https://github.com/leksious/HiFi-GAN.git

Cloning into 'HiFi-GAN'...


In [ ]:
#!g1.1
%%bash
mkdir ./data
mkdir ./saved
mkdir ./pretrained
wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 -O ./data/LJSpeech-1.1.tar.bz2
tar -C ./data -xjf ./data/LJSpeech-1.1.tar.bz2
rm ./data/LJSpeech-1.1.tar.bz2

In [169]:
#!g1.1
from HiFiGAN.configs import MPDConfig, MSDConfig, GeneratorConfig, MelSpectrogramConfig, TrainerConfig

In [170]:
#!g1.1
from HiFiGAN import trainer

In [171]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")

In [172]:
#!g1.1
trainer.trainer(GeneratorConfig, MPDConfig, MSDConfig, MelSpectrogramConfig, TrainerConfig)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lexious. Use `wandb login --relogin` to force relogin


100%|██████████| 90/90 [12:32:11<00:00, 501.46s/it]  


## Inference of the model

In [173]:
#!g1.1
from HiFiGAN.Generator import Generator  

In [174]:
#!g1.1
from HiFiGAN.configs import MPDConfig, MSDConfig, GeneratorConfig, MelSpectrogramConfig, TrainerConfig

In [175]:
#!g1.1
gen = Generator(GeneratorConfig)

In [176]:
#!g1.1
import torch

In [177]:
#!g1.1
chckp = torch.load('saves/best.pt')

In [178]:
#!g1.1
chckp = chckp['generator']

In [179]:
#!g1.1
gen.load_state_dict(chckp)

<All keys matched successfully>

In [180]:
#!g1.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [181]:
#!g1.1
gen.eval()

Generator(
  (p_c): Conv1d(80, 256, kernel_size=(7,), stride=(1,), padding=(3,))
  (usb_block): Sequential(
    (0): USBlock(
      (convolution): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
      (mrf): MRF(
        (blocks): ModuleList(
          (0): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            )
          )
          (1): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
              (1): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(12,), dilation=(6,))
            )
          )
          (2): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (1): C

In [182]:
#!g1.1
gen.to(device)

Generator(
  (p_c): Conv1d(80, 256, kernel_size=(7,), stride=(1,), padding=(3,))
  (usb_block): Sequential(
    (0): USBlock(
      (convolution): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
      (mrf): MRF(
        (blocks): ModuleList(
          (0): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            )
          )
          (1): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
              (1): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(12,), dilation=(6,))
            )
          )
          (2): ResBlock(
            (convolutions): ModuleList(
              (0): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (1): C

In [183]:
#!g1.1
import torchaudio
from HiFiGAN.utils import MelSpectrogram, MelSpectrogramConfig
from scipy.io.wavfile import write

In [184]:
#!g1.1
import wandb

In [185]:
#!g1.1
wandb.init(project='HiFi GAN')
melspec = MelSpectrogram(MelSpectrogramConfig).to(device)
wavs = ['audio_1.wav', 'audio_2.wav', 'audio_3.wav']
i = 0
for wave in wavs:
    wav_real = torchaudio.load(wave)[0].to(device)
    mel = melspec(wav_real)
    wav = gen(mel)
    mel_gen = melspec(wav)
    wav = wav.squeeze().cpu().detach().numpy()
    write('test' + str(i) + '.wav' , 22050, wav)
    i += 1
    audio = wandb.Audio(
        wav,
        sample_rate=22050
    )
    
    wandb.log({
        'audio': audio,
        'GT_Mel': wandb.Image(mel.detach().cpu()),
        'Gen_Mel': wandb.Image(mel_gen.detach().cpu())
    })

Discriminator Learning Rate,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
Discriminator Loss Train,█▇▆▄▄▃▆▃▃▃▃▃▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
FM Loss train,▆▁▂█▅▆▄▇▆▇▅▇▆▇▇▆▇▇▇▇▆▇▆▇▇▇▆▇█▇█▇▇▇▇▆█▇▇▇
Gen Loss on train,▁▂▄▇▆▅▅▅▆▆▇▇▇▇█▇███▇▇▇████▇█▇████▇███▇▇▇
Generator Loss train,█▇▆▅▄▄▃▃▃▄▃▃▃▃▃▂▃▂▂▃▂▂▂▃▃▂▂▂▂▁▁▁▂▂▂▁▂▁▁▁
LR generator,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
Loss Train,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss_val,█▆▅▅▄▄▃▃▃▃▃▂▂▂▃▂▂▂▂▃▂▁▂▂▁▁▁▄▂▁▁▁▁▁▁▁▂▂▂▁
Mel Loss Val,█▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▂▁▂▁▂▁
Mel Loss train,█▇▆▄▄▄▄▃▃▄▃▃▃▃▃▂▃▂▂▃▂▂▂▃▂▂▂▂▂▁▁▁▂▂▂▁▂▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [ ]:
#!g1.1
